In [ ]:
!pip install transformers accelerate torch huggingface_hub 
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import create_repo, upload_file

embedding_model_name = "em_model"
qa_model_name = "qa_model"

embedding_model = AutoModel.from_pretrained(embedding_model_name, load_in_4bit=True)
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

qa_model = AutoModel.from_pretrained(qa_model_name, load_in_4bit=True)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)

qa_model = qa_model.half().to("cuda")

save_directory_embedding = "./quantized_em_model"
embedding_model.save_pretrained(save_directory_embedding)
embedding_tokenizer.save_pretrained(save_directory_embedding)

save_directory = "./quantized_qa_model"
qa_model.save_pretrained(save_directory)
qa_tokenizer.save_pretrained(save_directory)

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!pip install -r /kaggle/working/llama.cpp/requirements/requirements-convert_hf_to_gguf.txt

!python /kaggle/working/llama.cpp/convert_hf_to_gguf.py /kaggle/working/quantized_qa_model --outfile qa.gguf --outtype auto
!python /kaggle/working/llama.cpp/convert_hf_to_gguf.py /kaggle/working/quantized_em_model --outfile em.gguf --outtype auto

In [ ]:
repo_name = "SurgicalMasks_Qwen2.5-3B-Instruct-4bit"
repo_name_embedding = "SurgicalMasks_bge-large-en-v1.5"
create_repo(repo_name, token="TOKEN")
create_repo(repo_name_embedding, token="TOKEN")

upload_file(
    path_or_fileobj="qa.gguf",
    path_in_repo="model",
    repo_id=repo_name,
    repo_type="model",
    token="TOKEN"
)
upload_file(
    path_or_fileobj="em.gguf",
    path_in_repo="model",
    repo_id=repo_name_embedding,
    repo_type="model",
    token="TOKEN"
)

